In [2]:
%load_ext lab_black
%cd ../../..

/home/shim/cev/dl/log-analytics


In [3]:
import argparse
import math
import multiprocessing
import sys
from datetime import datetime
from pathlib import Path
from pprint import pformat

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_optimizer
import yaml
from easydict import EasyDict
from pytorch_transformers import DistilBertForSequenceClassification, DistilBertTokenizer

from datasets.dataset_ver8 import DatasetGeneratorVer8
from utils import SAM, AverageMeter, CustomLogger, FocalLoss, seed_everything

from main_ver8 import MyTrainer
from collections import defaultdict
import matplotlib.pyplot as plt
import random
from sklearn.metrics import f1_score, classification_report
from tqdm import tqdm

In [4]:
postfix = "distilbert-base-uncased-focal-AdamW-lr1e-05-ver8-os10_4"
outdir = Path("results/distilbert-base-uncased-ver8")
fold = 4

In [5]:
with open("config/distilbert-base-uncased-ver8.yaml", "r") as f:
    C = EasyDict(yaml.load(f, yaml.FullLoader))
    C.result_dir = Path(C.result_dir)
    C.dataset.dir = Path(C.dataset.dir)
    seed_everything(C.seed, deterministic=False)

In [6]:
C

{'model': {'name': 'distilbert-base-uncased'},
 'comment': None,
 'result_dir': PosixPath('results/distilbert-base-uncased-ver8'),
 'debug': False,
 'seed': 20210425,
 'ver': 8,
 'train': {'SAM': False,
  'folds': [1],
  'checkpoints': [None],
  'loss': {'name': 'focal',
   'params': {'gamma': 2.0, 's': 45.0, 'm': 0.1, 'crit': 'focal'}},
  'optimizer': {'name': 'AdamW'},
  'finetune': {'do': True, 'step1_epochs': 2, 'step2_epochs': 4},
  'max_epochs': 12,
  'lr': 1e-05,
  'scheduler': {'name': 'ReduceLROnPlateau',
   'params': {'factor': 0.5, 'patience': 3, 'verbose': True}}},
 'dataset': {'dir': PosixPath('data/ver6'),
  'batch_size': 15,
  'num_workers': 8,
  'oversampling': True,
  'oversampling_scale': 10,
  'augmentation': {'EDA_RD': True}}}

In [7]:
trainer = MyTrainer(C, fold, outdir / f"{postfix}.pth")

Load pretrained results/distilbert-base-uncased-ver8/distilbert-base-uncased-focal-AdamW-lr1e-05-ver8-os10_4.pth


In [8]:
model = trainer.model
model.eval()
torch.set_grad_enabled(False)

In [9]:
activation = []


def hook(model, input, output):
    activation.append(output.detach().cpu())

In [19]:
model.module.pre_classifier.register_forward_hook(hook)

In [20]:
train_dl = trainer.dsgen.train_only()

# 덱만들기 (Train)

In [11]:
activation = []
deck = {"feat": [], "otext": [], "tlevel": [], "fclevel": []}
with tqdm(total=len(train_dl.dataset), ncols=100, file=sys.stdout) as t:
    for text, tlevel, otext in train_dl:
        fclevel = model(text.cuda(non_blocking=True))[0].argmax(dim=1).cpu()
        deck["fclevel"].append(fclevel)
        deck["tlevel"].append(tlevel)
        deck["otext"].extend(otext)
        t.update(text.size(0))

100%|██████████████████████████████████████████████████████| 421079/421079 [21:29<00:00, 326.65it/s]


In [12]:
deck["feat"] = torch.cat(activation)
deck["tlevel"] = torch.cat(deck["tlevel"])
deck["fclevel"] = torch.cat(deck["fclevel"])

In [13]:
deck["tlevel"].shape, deck["fclevel"].shape

(torch.Size([421079]), torch.Size([421079]))

In [14]:
deck["feat"].shape

torch.Size([421079, 768])

In [15]:
torch.save(deck, outdir / f"{postfix}-deck1.pth")

# Valid Lv7에 대한 결과 분석

In [15]:
deck = torch.load("results/distilbert-base-uncased-ver8/distilbert-base-uncased-focal-AdamW-lr1e-05-ver8-os10_4-deck1.pth")

In [22]:
ds_lv7 = trainer.dsgen.valid_lv7().dataset

In [23]:
text, otext = ds_lv7[0]
activation = []
print(model(text[None].cuda())[0].cpu())

tensor([[ -4.0167,   7.1360, -12.2478,  -5.6769, -14.3561,  -5.8472, -13.5847]])


In [24]:
dists, indices = torch.norm(deck["feat"] - activation[0][None], p=None, dim=1).topk(4, largest=False)
dists, indices, deck["tlevel"][indices]

(tensor([[43.4501, 44.8983, 51.9252, 55.7211]]),
 tensor([[442, 254, 629,  43]]),
 tensor([[0, 0, 0, 0]]))

In [25]:
text, otext = ds_lv7[1]
activation = []
print(model(text[None].cuda())[0].cpu())

tensor([[-3.3274, -2.9708, -9.1343,  1.5720, -9.5865, -1.5778, -9.2321]])


In [26]:
dists, indices = torch.norm(deck["feat"] - activation[0][None], p=None, dim=1).topk(4, largest=False)
dists, indices, deck["tlevel"][indices]

(tensor([[44.7579, 45.4656, 48.4418, 57.4916]]),
 tensor([[248, 442, 626, 558]]),
 tensor([[0, 0, 0, 0]]))

In [27]:
text, otext = ds_lv7[2]
activation = []
print(model(text[None].cuda())[0].cpu())

tensor([[  4.1614,   0.2562, -12.4453,  -5.0171, -14.5411,  -5.2203, -13.4889]])


In [28]:
dists, indices = torch.norm(deck["feat"] - activation[0][None], p=None, dim=1).topk(4, largest=False)
dists, indices, deck["tlevel"][indices]

(tensor([[45.2554, 52.6130, 58.8965, 59.3427]]),
 tensor([[254, 626, 255, 518]]),
 tensor([[0, 0, 0, 0]]))

# 덱만들기 (Submission)

In [29]:
ds_test = trainer.dsgen.test()

In [30]:
activation = []
deck = {"feat": [], "otext": [], "fclevel": [], "ids": []}
with tqdm(total=len(ds_test), ncols=100, file=sys.stdout) as t:
    for i in range(len(ds_test)):
        text, otext, ids = ds_test[i]
        fclevel = model(text[None].cuda(non_blocking=True))[0].argmax(dim=1).cpu()
        deck["fclevel"].append(fclevel)
        deck["otext"].append(otext)
        deck["ids"].append(ids)
        t.update()

100%|██████████████████████████████████████████████████| 1095951/1095951 [1:51:55<00:00, 163.20it/s]


In [31]:
deck["feat"] = torch.stack(activation)
deck["fclevel"] = torch.stack(deck["fclevel"])

In [32]:
deck["feat"] = deck["feat"][:, 0, :]
deck["fclevel"] = deck["fclevel"][:, 0]

In [33]:
deck["feat"].shape, deck["fclevel"].shape

(torch.Size([1095951, 768]), torch.Size([1095951]))

In [34]:
torch.save(deck, outdir / f"{postfix}-deck2.pth")